In [38]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [39]:
df = pd.read_excel("Updated_Data_Historis_2015_2025.xlsx", parse_dates=['Tanggal']) #parse = untuk ngubah kolom Tanggal jadi datetime
df = df.sort_values('Tanggal')
df = df.reset_index(drop=True)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4020 non-null   datetime64[ns]
 1   Temperatur Minimum                  4020 non-null   object        
 2   Temperatur Maksimum                 4020 non-null   object        
 3   Temperatur Rata-rata                4020 non-null   object        
 4   Kelembapan Rata-rata                4020 non-null   object        
 5   Curah Hujan (mm)                    4020 non-null   object        
 6   Lamanya Penyinaran Matahari         4020 non-null   object        
 7   Kecepatan Angin Maksimum            4020 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4020 non-null   int64         
 9   Kecepatan Angin Rata-rata           4020 non-null   int64         
 10  Arah Angin Terbanyak (°)

In [41]:
checkstripvalue = (df['Curah Hujan (mm)'] == "-").sum()
print(checkstripvalue)

497


In [42]:
df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].replace('-',0)

df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].astype(float)

C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_14568\2154469297.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].replace('-',0)


In [43]:
checkstripvalue2 = (df['Curah Hujan (mm)'] == 8888).sum()
print(checkstripvalue2)

209


# Fill Missing Values using RF Imputer

In [44]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

df2 = pd.read_excel(
    "Updated_Data_Historis_2015_2025.xlsx",
    parse_dates=["Tanggal"]
)
df2 = df2.sort_values("Tanggal").reset_index(drop=True)

invalid_values = ["-", " - ", "–", "—", "N/A", "n/a", "", " ", "8888", 8888]
df2 = df2.replace(invalid_values, np.nan)

# Tangkap SEMUA blank aneh dari Excel
df2 = df2.replace(r'^\s*$', np.nan, regex=True)

col = "Arah Angin Terbanyak (°)"

df2[col] = (
    df2[col].astype(str)
            .str.strip()
            .str.upper()
            .str.replace("–", "-", regex=False)
            .str.replace("—", "-", regex=False)
)

mapping = {
    "C": 0, "N": 0, "NE": 45, "E": 90, "SE": 135,
    "S": 180, "SW": 225, "W": 270, "NW": 315
}

df2[col] = df2[col].map(mapping)


C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_14568\4093600459.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2 = df2.replace(invalid_values, np.nan)


In [45]:
# 1. Pastikan hanya mengambil kolom numerik
numeric_df = df2.select_dtypes(include=[np.number])

# 2. Hitung korelasi terhadap kolom target saja
target_corr = numeric_df.corr()['Curah Hujan (mm)'].sort_values(ascending=False)

print(target_corr)

Curah Hujan (mm)                      1.000000
Kelembapan Rata-rata                  0.399640
Arah Angin Saat Kecepatan Maksimum    0.085265
Arah Angin Terbanyak (°)              0.002441
Kecepatan Angin Maksimum             -0.009466
Kecepatan Angin Rata-rata            -0.032244
Lamanya Penyinaran Matahari          -0.199244
Temperatur Maksimum                  -0.319690
Temperatur Rata-rata                 -0.388173
Temperatur Minimum                   -0.402973
Name: Curah Hujan (mm), dtype: float64


In [46]:
for c in df2.columns:
    if c != "Tanggal":
        df2[c] = pd.to_numeric(df2[c], errors="coerce")

target = "Curah Hujan (mm)"

feature_cols = [
    c for c in df2.columns
    if c not in ["Tanggal", target]
]
df2[feature_cols] = df2[feature_cols].fillna(
    df2[feature_cols].median()
)


train_df = df2[df2[target].notna()]
pred_df  = df2[df2[target].isna()]

X_train = train_df[feature_cols]
y_train = train_df[target]

X_pred = pred_df[feature_cols]

rf = RandomForestRegressor(
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

df2.loc[df2[target].isna(), target] = rf.predict(X_pred)

df2.to_excel("Ready_To_Use_Updated_RF.xlsx", index=False)
print("File disimpan: Ready_To_Use_Updated_RF.xlsx")

File disimpan: Ready_To_Use_Updated_RF.xlsx


In [47]:
data1 = pd.read_excel('Ready_To_Use_Updated_RF.xlsx')

In [48]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4020 non-null   datetime64[ns]
 1   Temperatur Minimum                  4020 non-null   float64       
 2   Temperatur Maksimum                 4020 non-null   float64       
 3   Temperatur Rata-rata                4020 non-null   float64       
 4   Kelembapan Rata-rata                4020 non-null   int64         
 5   Curah Hujan (mm)                    4020 non-null   float64       
 6   Lamanya Penyinaran Matahari         4020 non-null   float64       
 7   Kecepatan Angin Maksimum            4020 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4020 non-null   int64         
 9   Kecepatan Angin Rata-rata           4020 non-null   int64         
 10  Arah Angin Terbanyak (°)

In [49]:
data1.isna().sum().sort_values(ascending=False)


Tanggal                               0
Temperatur Minimum                    0
Temperatur Maksimum                   0
Temperatur Rata-rata                  0
Kelembapan Rata-rata                  0
Curah Hujan (mm)                      0
Lamanya Penyinaran Matahari           0
Kecepatan Angin Maksimum              0
Arah Angin Saat Kecepatan Maksimum    0
Kecepatan Angin Rata-rata             0
Arah Angin Terbanyak (°)              0
dtype: int64

# Feature Engineering

In [50]:
df3 = pd.read_excel("Ready_To_Use_Updated_RF.xlsx")

# Pastikan datetime & sort
df3['Tanggal'] = pd.to_datetime(df3['Tanggal'])
df3 = df3.sort_values('Tanggal').reset_index(drop=True)

df3['Curah Hujan Old'] = df3['Curah Hujan (mm)'].copy()

lags = [1, 3, 7]

lag_columns = [
    "Curah Hujan (mm)",
    "Kelembapan Rata-rata",
    "Temperatur Rata-rata",
    "Temperatur Maksimum",
    "Temperatur Minimum",
    "Lamanya Penyinaran Matahari",
    "Kecepatan Angin Rata-rata"
]

# STEP 1: Buat lag features dulu
for col in lag_columns:
    for lag in lags:
        df3[f"{col}_lag_{lag}"] = df3[col].shift(lag)

# STEP 2: Buat rolling mean DARI data yang sudah di-shift(1)
# Rolling mean untuk Curah Hujan
df3["Rain_roll_3"] = df3["Curah Hujan (mm)"].shift(1).rolling(3).mean()
df3["Rain_roll_7"] = df3["Curah Hujan (mm)"].shift(1).rolling(7).mean()

# Rolling mean untuk Kelembapan Rata-rata
df3["Humidity_roll_3"] = df3["Kelembapan Rata-rata"].shift(1).rolling(3).mean()
df3["Humidity_roll_7"] = df3["Kelembapan Rata-rata"].shift(1).rolling(7).mean()

# Rolling mean untuk Temperatur Rata-rata
df3["Temp_Avg_roll_3"] = df3["Temperatur Rata-rata"].shift(1).rolling(3).mean()
df3["Temp_Avg_roll_7"] = df3["Temperatur Rata-rata"].shift(1).rolling(7).mean()

# Rolling mean untuk Temperatur Maksimum
df3["Temp_Max_roll_3"] = df3["Temperatur Maksimum"].shift(1).rolling(3).mean()
df3["Temp_Max_roll_7"] = df3["Temperatur Maksimum"].shift(1).rolling(7).mean()

# Rolling mean untuk Temperatur Minimum
df3["Temp_Min_roll_3"] = df3["Temperatur Minimum"].shift(1).rolling(3).mean()
df3["Temp_Min_roll_7"] = df3["Temperatur Minimum"].shift(1).rolling(7).mean()

# Rolling mean untuk Lamanya Penyinaran Matahari
df3["Sunshine_roll_3"] = df3["Lamanya Penyinaran Matahari"].shift(1).rolling(3).mean()
df3["Sunshine_roll_7"] = df3["Lamanya Penyinaran Matahari"].shift(1).rolling(7).mean()

# Rolling mean untuk Kecepatan Angin Rata-rata
df3["Wind_roll_3"] = df3["Kecepatan Angin Rata-rata"].shift(1).rolling(3).mean()
df3["Wind_roll_7"] = df3["Kecepatan Angin Rata-rata"].shift(1).rolling(7).mean()

# STEP 3: Buat EWM DARI data yang sudah di-shift(1)
df3['Rain_ewm_3'] = df3['Curah Hujan (mm)'].shift(1).ewm(span=3).mean()
df3['Rain_ewm_7'] = df3['Curah Hujan (mm)'].shift(1).ewm(span=7).mean()

# STEP 4: Buat rolling std DARI data yang sudah di-shift(1)
df3['Rain_roll_std_7'] = df3['Curah Hujan (mm)'].shift(1).rolling(7).std()
df3['Temp_roll_std_7'] = df3['Temperatur Rata-rata'].shift(1).rolling(7).std()

df3["month"] = df3["Tanggal"].dt.month
df3["dayofyear"] = df3["Tanggal"].dt.dayofyear

df3["sin_doy"] = np.sin(2 * np.pi * df3["dayofyear"] / 365)
df3["cos_doy"] = np.cos(2 * np.pi * df3["dayofyear"] / 365)

df3["Rain_3day_sum"] = df3["Curah Hujan (mm)"].shift(1).rolling(3).sum()
df3["Rain_7day_sum"] = df3["Curah Hujan (mm)"].shift(1).rolling(7).sum()

q90 = df3["Curah Hujan (mm)"].quantile(0.9)

df3["HeavyRain_Flag"] = (
    df3["Curah Hujan (mm)"].shift(1) > q90
).astype(int)


df3_fe = df3.dropna().reset_index(drop=True)

df3_fe.to_excel("df3_feature_engineered_lag137_fixed.xlsx", index=False)

In [51]:
data = pd.read_excel('df3_feature_engineered_lag137_fixed.xlsx')

In [52]:
# 1. Pastikan hanya mengambil kolom numerik
numeric_df2 = data.select_dtypes(include=[np.number])

# 2. Hitung korelasi terhadap kolom target saja
target_corr = numeric_df2.corr()['Curah Hujan Old'].sort_values(ascending=False)

print(target_corr)

Curah Hujan (mm)                      1.000000
Curah Hujan Old                       1.000000
Kelembapan Rata-rata                  0.397834
Kelembapan Rata-rata_lag_1            0.366699
Humidity_roll_3                       0.298581
Humidity_roll_7                       0.263789
Rain_ewm_7                            0.237430
Rain_ewm_3                            0.220119
Rain_roll_7                           0.207760
Rain_7day_sum                         0.207760
Rain_3day_sum                         0.202300
Rain_roll_3                           0.202300
Kelembapan Rata-rata_lag_3            0.191844
Curah Hujan (mm)_lag_1                0.179794
cos_doy                               0.175952
Rain_roll_std_7                       0.167932
Kelembapan Rata-rata_lag_7            0.147756
sin_doy                               0.142736
HeavyRain_Flag                        0.125471
Temp_roll_std_7                       0.102920
Curah Hujan (mm)_lag_3                0.092645
Curah Hujan (

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4013 entries, 0 to 4012
Data columns (total 58 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4013 non-null   datetime64[ns]
 1   Temperatur Minimum                  4013 non-null   float64       
 2   Temperatur Maksimum                 4013 non-null   float64       
 3   Temperatur Rata-rata                4013 non-null   float64       
 4   Kelembapan Rata-rata                4013 non-null   int64         
 5   Curah Hujan (mm)                    4013 non-null   float64       
 6   Lamanya Penyinaran Matahari         4013 non-null   float64       
 7   Kecepatan Angin Maksimum            4013 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4013 non-null   int64         
 9   Kecepatan Angin Rata-rata           4013 non-null   int64         
 10  Arah Angin Terbanyak (°)

# Data Modelling 

In [54]:
# from sklearn.model_selection import TimeSeriesSplit
# from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBRegressor

# df4 = pd.read_excel("df3_feature_engineered_lag137_fixed.xlsx")

# df4["Rain_Event"] = (df4["Curah Hujan Old"] > 0).astype(int)

# feature_cols = df4.columns.drop([
#     "Tanggal",
#     "Curah Hujan Old",
#     "Curah Hujan (mm)",
#     "Rain_Event",
#     "Kelembapan Rata-rata",
#     "Temperatur Rata-rata",
#     "Temperatur Maksimum",
#     "Temperatur Minimum",
#     "Lamanya Penyinaran Matahari",
#     "Kecepatan Angin Rata-rata"
    
# ])

# X = df4[feature_cols]
# y_cls = df4["Rain_Event"]

# tscv = TimeSeriesSplit(n_splits=5)
# train_idx, test_idx = list(tscv.split(X))[-1]

# X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
# y_train_cls, y_test_cls = y_cls.iloc[train_idx], y_cls.iloc[test_idx]

# # clf = RandomForestClassifier(
# #     n_estimators=300,
# #     max_depth=12,
# #     min_samples_leaf=5,
# #     class_weight={0: 1, 1: 2},
# #     random_state=42,
# #     n_jobs=-1
# # )


# clf = XGBClassifier(
#     n_estimators=400,
#     max_depth=4,
#     learning_rate=0.05,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     scale_pos_weight=(y_train_cls == 0).sum() / (y_train_cls == 1).sum(),
#     eval_metric="logloss",
#     random_state=42
# )


# clf.fit(X_train, y_train_cls)

# y_pred_prob = clf.predict_proba(X_test)[:, 1]

# rain_threshold = 0.3
# y_pred_cls = (y_pred_prob >= rain_threshold).astype(int)

# print(
#     "Stage 1 — Rain Event Accuracy:",
#     accuracy_score(y_test_cls, y_pred_cls)
# )

# df4_rain = df4[df4["Curah Hujan Old"] > 0].copy()

# X_rain = df4_rain[feature_cols]
# y_rain = np.log1p(df4_rain["Curah Hujan Old"])

# train_idx_r, test_idx_r = list(tscv.split(X_rain))[-1]

# X_train_r, X_test_r = X_rain.iloc[train_idx_r], X_rain.iloc[test_idx_r]
# y_train_r, y_test_r = y_rain.iloc[train_idx_r], y_rain.iloc[test_idx_r]

# reg = XGBRegressor(
#     n_estimators=800,
#     max_depth=6,
#     learning_rate=0.03,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     min_child_weight=5,
#     reg_alpha=0.5,
#     reg_lambda=1.5,
#     objective="reg:squarederror",
#     random_state=42
# )

# reg.fit(X_train_r, y_train_r)
# final_pred = []

# for i in range(len(X_test)):
#     prob = y_pred_prob[i]

#     if prob < rain_threshold:
#         final_pred.append(0)
#     else:
#         rain_log = reg.predict(X_test.iloc[[i]])[0]
#         rain_mm = np.expm1(rain_log)
#         final_pred.append(prob * rain_mm)

# final_pred = np.array(final_pred)

# y_true = df4.iloc[test_idx]["Curah Hujan Old"]

# mae = mean_absolute_error(y_true, final_pred)
# mse = mean_squared_error(y_true, final_pred)
# rmse = np.sqrt(mse)

# print("FINAL MAE  (Two-Stage):", mae)
# print("FINAL MSE  (Two-Stage):", mse)
# print("FINAL RMSE (Two-Stage):", rmse)


# result = df4.iloc[test_idx][["Tanggal"]].copy()
# result["Actual_Rain_mm"] = y_true.values
# result["Predicted_Rain_mm"] = final_pred

# result.to_excel("two_stage_prediction_result.xlsx", index=False)
# print("Selesai! Hasil tersimpan.")


In [55]:
# =========================================
# DATA MODELLING — TWO STAGE (UPGRADED)
# =========================================

import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

from xgboost import XGBClassifier, XGBRegressor

# =========================================
# LOAD DATA
# =========================================
df4 = pd.read_excel("df3_feature_engineered_lag137_fixed.xlsx")

# Binary rain event
df4["Rain_Event"] = (df4["Curah Hujan Old"] > 0).astype(int)

# =========================================
# FEATURE SELECTION (tetap seperti kamu)
# =========================================
feature_cols = df4.columns.drop([
    "Tanggal",
    "Curah Hujan Old",
    "Curah Hujan (mm)",
    "Rain_Event",
    "Kelembapan Rata-rata",
    "Temperatur Rata-rata",
    "Temperatur Maksimum",
    "Temperatur Minimum",
    "Lamanya Penyinaran Matahari",
    "Kecepatan Angin Rata-rata"
])

X = df4[feature_cols]
y_cls = df4["Rain_Event"]

# =========================================
# TIME SERIES SPLIT
# =========================================
tscv = TimeSeriesSplit(n_splits=5)
train_idx, test_idx = list(tscv.split(X))[-1]

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train_cls, y_test_cls = y_cls.iloc[train_idx], y_cls.iloc[test_idx]

# =========================================
# STAGE 1 — CLASSIFICATION (UPGRADED)
# =========================================
scale_pos_weight = (y_train_cls == 0).sum() / (y_train_cls == 1).sum()

clf = XGBClassifier(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    eval_metric="logloss",
    random_state=42
)

clf.fit(X_train, y_train_cls)

y_pred_prob = clf.predict_proba(X_test)[:, 1]

print(
    "Stage 1 — Rain Event Accuracy:",
    accuracy_score(y_test_cls, (y_pred_prob >= 0.5).astype(int))
)

# =========================================
# STAGE 2 — REGRESSION (HUJAN SAJA, UPGRADED)
# =========================================
df4_rain = df4[df4["Curah Hujan Old"] > 0].copy()

X_rain = df4_rain[feature_cols]
y_rain = np.log1p(df4_rain["Curah Hujan Old"])

train_idx_r, test_idx_r = list(tscv.split(X_rain))[-1]

X_train_r, X_test_r = X_rain.iloc[train_idx_r], X_rain.iloc[test_idx_r]
y_train_r, y_test_r = y_rain.iloc[train_idx_r], y_rain.iloc[test_idx_r]

# Sample weight → fokus hujan deras
sample_weight = np.clip(
    df4_rain["Curah Hujan Old"].values,
    1,
    np.percentile(df4_rain["Curah Hujan Old"], 90)
)

reg = XGBRegressor(
    n_estimators=1200,
    max_depth=7,
    learning_rate=0.02,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_weight=3,
    reg_alpha=0.3,
    reg_lambda=1.2,
    objective="reg:squarederror",
    random_state=42
)

reg.fit(
    X_train_r,
    y_train_r,
    sample_weight=sample_weight[train_idx_r]
)

y_true = df4.iloc[test_idx]["Curah Hujan Old"].values

thresholds = np.arange(0.05, 0.61, 0.05)
best_thr = None
best_mae = np.inf

for thr in thresholds:
    temp_pred = []

    for i in range(len(X_test)):
        prob = y_pred_prob[i]

        if prob < thr:
            temp_pred.append(0)
        else:
            rain_log = reg.predict(X_test.iloc[[i]])[0]
            rain_mm = np.expm1(rain_log)

            # Remove damping for heavy rain
            if rain_mm > 50:
                temp_pred.append(rain_mm)
            else:
                temp_pred.append(prob * rain_mm)

    temp_pred = np.array(temp_pred)
    mae = mean_absolute_error(y_true, temp_pred)

    if mae < best_mae:
        best_mae = mae
        best_thr = thr

print("Best Threshold:", best_thr)
print("Best MAE:", best_mae)

final_pred = []

for i in range(len(X_test)):
    prob = y_pred_prob[i]

    if prob < best_thr:
        final_pred.append(0)
    else:
        rain_log = reg.predict(X_test.iloc[[i]])[0]
        rain_mm = np.expm1(rain_log)

        if rain_mm > 50:
            final_pred.append(rain_mm)
        else:
            final_pred.append(prob * rain_mm)

final_pred = np.array(final_pred)

mae = mean_absolute_error(y_true, final_pred)
mse = mean_squared_error(y_true, final_pred)
rmse = np.sqrt(mse)

print("FINAL MAE  (Two-Stage):", mae)
print("FINAL MSE  (Two-Stage):", mse)
print("FINAL RMSE (Two-Stage):", rmse)


result = df4.iloc[test_idx][["Tanggal"]].copy()
result["Actual_Rain_mm"] = y_true
result["Predicted_Rain_mm"] = final_pred

result.to_excel("two_stage_prediction_result_UPGRADED.xlsx", index=False)
print("Selesai! Hasil tersimpan (UPGRADED).")


Stage 1 — Rain Event Accuracy: 0.6796407185628742
Best Threshold: 0.6000000000000001
Best MAE: 6.175094877070448
FINAL MAE  (Two-Stage): 6.175094877070448
FINAL MSE  (Two-Stage): 230.9528951287857
FINAL RMSE (Two-Stage): 15.197134438070412
Selesai! Hasil tersimpan (UPGRADED).
